In [1]:
from pattern import web
import requests
from bs4 import BeautifulSoup
import doctest
import re
import pandas as pd
import sys

In [2]:
def getPages(url):
    '''
    Function
    --------
    Returns the page range of the query
    
    Parameters
    ----------
    url: str
        First page of the British Museum query
        
    Returns
    -------
    A tuple containing the first page and the last 
    page numbers, respectively.
    
    Example
    -------
    For "http://www.britishmuseum.org/research/collection_online/search.aspx?searchText=augustus"
    it would return (1, 474)
    
    Doctests
    --------
    #Regular Case
    >>> getPages('http://www.britishmuseum.org/research/collection_online/search.aspx?searchText=augustus')
    (1, 474)
    
    #Only one results page
    >>> getPages('http://www.britishmuseum.org/research/collection_online/search.aspx?searchText=1855,0512.40')
    (1, 1)
    '''
    #get page and convert to web.Element object
    html = requests.get(url).text
    dom = web.Element(html)
    
    #finds the list of pages and returns (1, 1) if no list of pages present
    elem = dom.by_class('colSearchPaging')
    if len(elem) < 1:
        return (1, 1)
    else:
        pages = elem[0].children
    
    #gets the last page number of the pages list
    last_page = pages[-4].children[0].content
    
    return (1, int(last_page))
    
#doctest.testmod()

In [3]:
def getLinks(url):
    '''
    Function
    --------
    Gets the urls to the objects from results page
    
    Parameter
    ---------
    url: str
        Search page of British Museum website
        
    Returns
    -------
    A list of str that contain the links of the objects
    
    Doctest
    -------
    >>> getLinks('http://www.britishmuseum.org/research/collection_online/search.aspx?searchText=1855,0512.40')
    [u'http://www.britishmuseum.org/research/collection_online/collection_object_details.aspx?objectId=1128207&partId=1&searchText=1855%2c0512.40&page=1', u'http://www.britishmuseum.org/research/collection_online/collection_object_details.aspx?objectId=3629209&partId=1&searchText=1855%2c0512.40&page=1']
    '''
    links = []
    
    #get page and convert to web.Element object
    html = requests.get(url).text
    dom = web.Element(html)
    
    #create list of row objects
    rows = dom.by_class('grid_12 alpha row colResults')
    
    #traverse each row and get the links
    for row in rows:
        objects = row.by_class('noImage') + row.by_class('image')
        for obj in objects:
            link_identifier_with_noise = obj.attr['href'].split('/')[-1]
            link_identifier = link_identifier_with_noise[5:]
            link = 'http://www.britishmuseum.org/research/collection_online/' + link_identifier
            links.append(link)
    
    return links

doctest.testmod()

TestResults(failed=0, attempted=3)

In [4]:
def getDetails(url):
    '''
    Function
    --------
    Reads the data off of the given British Museum item page and puts into a dictionary
    
    Parameter
    ---------
    url: str
        The url of the British Museum item page
        
    Returns
    -------
    A dictionary with of result under 'result' key if reached page, else puts url under
    'skipped' key
    '''
    result = {}
    skipped = ''
    #create list of descriptors
    descriptors = {'Museum number', 'Denomination', 'Description', 'State', 
                 'Authority', 'Culture/period', 'Date', 'Materials', 
                  'Production place', 'Curator\'s comments', 'Bibliography', 
                  'Subjects', 'Associated names', 'Object type', 'Weight'}
    
    #get page and convert to web.Element object
    html = requests.get(url).text
    dom = web.Element(html)
    
    #gets portion of page containing discription
    try:
        details = dom.by_class('objectDetails')[0].children
    except:
        details = []
        skipped = url
    
    #iterate over and populate the result dictionary
    for detail in details:
        detail = list(BeautifulSoup(str(detail), 'html.parser').stripped_strings)
        #print(detail)
        if len(detail) > 1:
            desc = detail[0]
            if desc in descriptors:
                result[desc] = detail[1]
            elif desc == 'Dimensions':
                for dim in detail[1:]:
                    split_dim = dim.split(': ')
                    try: 
                        if split_dim[0] in 'Weight':
                            result['Weight (g)'] = re.findall("\d+\.\d+", split_dim[1])[0]
                    except:
                        pass
            elif desc == 'Inscriptions':
                inscriptions = []
                i = 1
                inscription = {}
                while i < len(detail):
                    if 'Inscription ' in detail[i]:
                        if detail[i] == 'Inscription Type':
                            if inscription != {}:
                                inscriptions.append(inscription)
                            inscription = {}
                        descriptor = detail[i]
                        i += 1
                        description = detail[i]
                        i += 1
                        try:
                            while 'Inscription ' not in detail[i]:
                                description += ' ' + detail[i]
                                i += 1
                        except:
                            pass
                        inscription[descriptor] = description
                    else:
                        i += 1
                result[desc] = inscriptions
            
    return {'result': result, 'skipped': skipped}
              
              
#getDetails('')

In [5]:
url = 'http://www.britishmuseum.org/research/collection_online/search.aspx?searchText=augustus&lookup-people=e.g.+Hokusai%2C+Ramesses&people=&lookup-place=e.g.+India%2C+Shanghai%2C+Thebes&place=&from=bc&fromDate=44&to=ad&toDate=14&lookup-object=coins&object=&lookup-subject=e.g.+farming%2C+New+Testament&subject=&lookup-matcult=e.g.+Choson+Dynasty%2C+Ptolemaic&matcult=&lookup-technique=e.g.+carved%2C+celadon-glazed&technique=&lookup-school=e.g.+French%2C+Mughal+Style&school=&lookup-material=e.g.+canvas%2C+porcelain%2C+silk&material=&lookup-ethname=e.g.+Hmong%2C+Maori%2C+Tai&ethname=&lookup-ware=e.g.+Imari+ware%2C+Qingbai+ware&ware=&lookup-escape=e.g.+cylinder%2C+gravity%2C+lever&escape=&lookup-bibliography=&bibliography=&citation=&museumno=&catalogueOnly=&view='
url = url + '&page='

In [6]:
pages = getPages(url)
links = [getLinks(url + str(i)) for i in range(1, pages[1]+1)]
links = [item for sublist in links for item in sublist]
data = [getDetails(link) for link in links]
details = []
for datum in data:
    details.append(datum['result'])
    skipped = datum['skipped']
    while skipped:
        result = getDetails(skipped)
        if result['result']:
            details.append(result['result'])
            skipped = ''
            

In [9]:
len(details)

3298

In [19]:
df = pd.DataFrame(details)
df.to_csv('AugustusCoins_44BC-14AD.csv', encoding='utf-8')